In [3]:
# ================================
# 1. Imports & basic configuration
# ================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from scipy.ndimage import median_filter
from scipy.signal import find_peaks

plt.style.use("default")  # keep plots clean

# Path to the Dips + Pull-ups CSV file (relative to this notebook)
DATA_PATH = Path("..") / "data" / "Dips_PU.csv"

print("Using data file:", DATA_PATH)

# ENMO smoothing parameters
MEDIAN_FILTER_SIZE = 5          # samples
MOVING_AVERAGE_WINDOW_S = 0.2   # seconds

# Dips & pull-ups repetition detection parameters
EXERCISE_PARAMS = {
    "Dips": {
        "min_interval_s": 1.3,   # dips are faster
        "percentile": 88,        # amplitude threshold (adaptive)
    },
    "Pull-ups": {
        "min_interval_s": 2.0,   # pull-ups slower
        "percentile": 85,
    },
}


Using data file: ..\data\Dips_PU.csv
